IF NO STARTUP SCRIPT WAS USED, RUN IN TERMINAL:

<code>git clone https://github.com/chris-ernst/text2video-workstation .</code>

Then create and activate a conda environment:

<code>

conda create --prefix /home/python3.9 python=3.9 ipykernel mamba -y

conda init bash
source .bashrc

conda activate /home/python3.9/

python -m ipykernel install --user --name=python3.9

</code>

Install necessary packages:

<code>

pip3 install diffusers transformers accelerate scipy safetensors mediapy accelerate stable_diffusion_videos

pip3 install psycopg2-binary pyuploadcare gradio python-dotenv ipywidgets

apt-get update && apt-get install libgl1

apt install ffmpeg

</code>

In [4]:
import ipywidgets as widgets

w = widgets.RadioButtons(
    options =["none","stabilityai/stable-diffusion-2-1", "stabilityai/stable-diffusion-2-base", "runwayml/stable-diffusion-v1-5", "SG161222/Realistic_Vision_V1.3", "prompthero/openjourney", "jzli/DreamShaper-3.32", "nitrosocke/redshift-diffusion","nitrosocke/elden-ring-diffusion", "nitrosocke/Future-Diffusion", "nitrosocke/mo-di-diffusion"],
    description='Model to be installed:',
    disabled=False
)

display(w)

RadioButtons(description='Model to be installed:', options=('none', 'stabilityai/stable-diffusion-2-1', 'stabi…

In [5]:
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

from stable_diffusion_videos import StableDiffusionWalkPipeline
import torch
import mediapy as media

import os
from dotenv import load_dotenv
from pyuploadcare import Uploadcare, File
import psycopg2

load_dotenv()

UC_public_key = os.getenv('UPLOADCARE_PUBLIC_KEY')
UC_secret_key = os.getenv('UPLOADCARE_PRIVATE_KEY')

def get_db_connection():
    conn = psycopg2.connect(
        dbname=os.getenv('DB_DBNAME'),
        user=os.getenv('DB_USER'),
        password=os.getenv('DB_PASSWORD'),
        host=os.getenv('DB_HOST'),
        port=os.getenv('DB_PORT'),
    )
    return conn

model_id = w.value
print(model_id)

if model_id == "none":
    print("no model defined")
if model_id == "stabilityai/stable-diffusion-2-1":
    pipeline = StableDiffusionWalkPipeline.from_pretrained(
        model_id,
        revision="fp16",
        torch_dtype=torch.float16,
    ).to("cuda")
if model_id == "stabilityai/stable-diffusion-2-base":
    pipeline = StableDiffusionWalkPipeline.from_pretrained(
        model_id,
        revision="fp16",
        torch_dtype=torch.float16,
    ).to("cuda")
else: 
    pipeline = StableDiffusionWalkPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
    ).to("cuda")



nitrosocke/redshift-diffusion


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
The config attributes {'scaling_factor': 0.18215} were passed to AutoencoderKL, but are not expected and will be ignored. Please verify your config.json configuration file.


In [6]:
video_path = pipeline.walk(
    prompts=['redshift style, hyperrealistic portrait of two stunning young women wearing intricate futuristic cyberpunk augmentations, highly detailed, punk hairstyle, 50mm, close up, masterpiece, photography, fog in the background, stunning lighting', 
             'redshift style, hyperrealistic portrait of two stunning 30 year old women wearing intricate futuristic cyberpunk augmentations, highly detailed, punk hairstyle, 50mm, close up, masterpiece, photography, fog in the background, stunning lighting'],
    seeds=[2496794, 2496794],
    num_interpolation_steps=3,
    height=512,  # use multiples of 64 if > 512. Multiples of 8 if < 512.
    width=1024,   # use multiples of 64 if > 512. Multiples of 8 if < 512.
    output_dir='outputs',        # Where images/videos will be saved
    name='test5',        # Subdirectory of output_dir where images/videos will be saved
    guidance_scale=8,         # Higher adheres to prompt more, lower lets model take the wheel
    num_inference_steps=50,     # Number of diffusion steps per image generated. 50 is good default
    upsample = False, 
    negative_prompt=""
)

video = media.read_video(video_path)
media.show_video(video, fps=6) 



  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

This browser does not support the video tag.


ValueError: public_key is required

In [ ]:
public_key = os.getenv('UPLOADCARE_PUBLIC_KEY')
secret_key = os.getenv('UPLOADCARE_PRIVATE_KEY')

uploadcare = Uploadcare(public_key, secret_key)
uploadcareFile = uploadcare.upload(video_path)
print(uploadcareFile)

with open(video_path, 'rb') as file_object:
    uc_file: File = uploadcare.upload(file_object, store=True)
    print(uc_file.info)
    print(uc_file.original_file_url)